In this Exercise you will use Decsion Tree and Logisit Regression for Multi classification
1. you will Need data file provided to you as "bezdekIris.data"
2. Note this Data file is uploaded in the following path but you can upload it any where inyour drive
3.drive/My Drive/Colab Notebooks/bezdekIris.data

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=ee716f2591429b7ad0766a0e1e17f8bc2c57e47d5f06014a05215f1dde02e708
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
  dataset = spark.read.csv('/content/drive/MyDrive/big data/week 3/bezdekIris.data',inferSchema=True, header =True)\
.toDF("sep_len", "sep_wid", "pet_len", "pet_wid", "label")

In [9]:
dataset.select('label').distinct().show(10)

+---------------+
|          label|
+---------------+
| Iris-virginica|
|    Iris-setosa|
|Iris-versicolor|
+---------------+



In [10]:
dataset.count()

149

In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
vector_assembly = VectorAssembler(\
inputCols = ["sep_len", "sep_wid", "pet_len", "pet_wid"],\
outputCol = 'features')
df_temp = vector_assembly.transform(dataset)
df_temp.show()                                  

+-------+-------+-------+-------+-----------+-----------------+
|sep_len|sep_wid|pet_len|pet_wid|      label|         features|
+-------+-------+-------+-------+-----------+-----------------+
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
|    5.4|    3.9|    1.7|    0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|
|    4.6|    3.4|    1.4|    0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|
|    5.0|    3.4|    1.5|    0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|
|    4.4|    2.9|    1.4|    0.2|Iris-setosa|[4.4,2.9,1.4,0.2]|
|    4.9|    3.1|    1.5|    0.1|Iris-setosa|[4.9,3.1,1.5,0.1]|
|    5.4|    3.7|    1.5|    0.2|Iris-setosa|[5.4,3.7,1.5,0.2]|
|    4.8|    3.4|    1.6|    0.2|Iris-setosa|[4.8,3.4,1.6,0.2]|
|    4.8|    3.0|    1.4|    0.1|Iris-setosa|[4.8,3.0,1.4,0.1]|
|    4.3|    3.0|    1.1|    0.1|Iris-se

In [17]:
df = df_temp.drop('sep_len', 'sep_wid', 'pet_len', 'pet_wid')

In [18]:
df.show(5)

+-----------+-----------------+
|      label|         features|
+-----------+-----------------+
|Iris-setosa|[4.9,3.0,1.4,0.2]|
|Iris-setosa|[4.7,3.2,1.3,0.2]|
|Iris-setosa|[4.6,3.1,1.5,0.2]|
|Iris-setosa|[5.0,3.6,1.4,0.2]|
|Iris-setosa|[5.4,3.9,1.7,0.4]|
+-----------+-----------------+
only showing top 5 rows



Now we should index labels, i.e., convert textual representation to a numeric
List item
List item
one with the help of StringIndexer. To do this, execute the following commands:

In [22]:
from pyspark.ml.feature import StringIndexer 
l_indexer = StringIndexer(inputCol='label',outputCol='labelIndex')
df=l_indexer.fit(df).transform(df)

In [25]:
df.select('label','labelIndex').distinct().show(6)

+---------------+----------+
|          label|labelIndex|
+---------------+----------+
|Iris-versicolor|       0.0|
| Iris-virginica|       1.0|
|    Iris-setosa|       2.0|
+---------------+----------+



After label indexing, we should divide our data into training and test sets (30% held out for testing):

In [26]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])

**Multi Classification using Decsion Tree
Explain what steps are taken for the Multi Classifications**

In [30]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import  MulticlassClassificationEvaluator
dt = DecisionTreeClassifier(labelCol='labelIndex',featuresCol='features',impurity='entropy',maxDepth=4,seed=1234)
model = dt.fit(trainingData)
predictions = model.transform(testData)

To estimate the accuracy of the prediction, the test error should be computed :
why we have MulticlassClassificationEvaluator???


In [40]:
evaluator = MulticlassClassificationEvaluator(\
labelCol='labelIndex', predictionCol='prediction',\
metricName='accuracy')
accuracy=evaluator.evaluate(predictions)
print("Test accuracy =  " , accuracy)
print(model.toDebugString)   

                                          

Test accuracy =   0.9285714285714286
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_a80dc7267abc, depth=4, numNodes=9, numClasses=3, numFeatures=4
  If (feature 2 <= 2.45)
   Predict: 2.0
  Else (feature 2 > 2.45)
   If (feature 3 <= 1.65)
    If (feature 2 <= 4.95)
     Predict: 0.0
    Else (feature 2 > 4.95)
     If (feature 0 <= 6.05)
      Predict: 0.0
     Else (feature 0 > 6.05)
      Predict: 1.0
   Else (feature 3 > 1.65)
    Predict: 1.0



In [41]:
# this is code for multiple classification using logistic Regression 
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LogisticRegression
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
lr = LogisticRegression(maxIter=100, \
                        
                        featuresCol="features", \
                        
                        labelCol='labelIndex')
ovr = OneVsRest(classifier=lr, \
                labelCol='labelIndex', \
                featuresCol='features')

In [43]:
ovrModel = ovr.fit(train)
predictionsovr = ovrModel.transform(test)


In [44]:
evaluator = MulticlassClassificationEvaluator(\
labelCol="labelIndex", predictionCol="prediction",\
metricName="accuracy")
accuracy = evaluator.evaluate(predictionsovr)
print("Test accuracy =  " , accuracy)

Test accuracy =   0.9361702127659575
